In [10]:
from collections import deque
import heapq
from bisect import bisect_left, bisect_right

# 1. Equipment Rental Availability (Graph - BFS)
def find_shortest_path(n, edges, availability, start_provider, target_equipment):
    graph = {i: [] for i in range(1, n + 1)}
    for a, b in edges:
        graph[a].append(b)
        graph[b].append(a)
    
    queue = deque([(start_provider, [start_provider])])
    visited = set([start_provider])
    
    while queue:
        provider, path = queue.popleft()
        if target_equipment in availability.get(provider, []):
            return path
        
        for neighbor in graph[provider]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, path + [neighbor]))
    
    return -1

n = 5
edges = [(1, 2), (2, 3), (3, 4), (4, 5)]
availability = {1: ["excavator"], 2: [], 3: ["bulldozer"], 4: ["excavator"], 5: ["crane"]}
start_provider = 2
target_equipment = "excavator"
print(find_shortest_path(n, edges, availability, start_provider, target_equipment))

[2, 1]


In [11]:
# 2. Optimal Equipment Deal Matching (Heap / Priority Queue)
def find_best_prices(requests, sellers):
    equipment_prices = {}
    
    for eq, price in sellers:
        if eq not in equipment_prices:
            equipment_prices[eq] = []
        heapq.heappush(equipment_prices[eq], price)
    
    result = []
    for eq, max_price in requests:
        if eq in equipment_prices:
            while equipment_prices[eq] and equipment_prices[eq][0] > max_price:
                heapq.heappop(equipment_prices[eq])
            result.append(equipment_prices[eq][0] if equipment_prices[eq] else None)
        else:
            result.append(None)
    
    return result

requests = [("excavator", 50000), ("bulldozer", 70000)]
sellers = [("excavator", 45000), ("bulldozer", 68000), ("excavator", 48000)]
print(find_best_prices(requests, sellers))

[45000, 68000]


In [13]:
# 3. Maintenance Log Analysis (Segment Tree)
class SegmentTree:
    def __init__(self, values):
        self.n = len(values)
        self.tree = [0] * (2 * self.n)
        for i in range(self.n):
            self.tree[self.n + i] = values[i]
        for i in range(self.n - 1, 0, -1):
            self.tree[i] = self.tree[2 * i] + self.tree[2 * i + 1]
    
    def query(self, left, right):
        left += self.n
        right += self.n
        total = 0
        while left < right:
            if left % 2:
                total += self.tree[left]
                left += 1
            if right % 2:
                right -= 1
                total += self.tree[right]
            left //= 2
            right //= 2
        return total

def process_maintenance_queries(maintenance_logs, queries):
    date_to_index = {date: i for i, date in enumerate(sorted(set(log[1] for log in maintenance_logs)))}
    costs = [0] * len(date_to_index)
    
    for eid, date, cost in maintenance_logs:
        costs[date_to_index[date]] += cost
    
    segment_tree = SegmentTree(costs)
    
    results = []
    for start_date, end_date in queries:
        l, r = bisect_left(list(date_to_index.keys()), start_date), bisect_right(list(date_to_index.keys()), end_date)
        results.append(segment_tree.query(l, r))
    
    return results

maintenance_logs = [(101, "2024-01-01", 500), (102, "2024-01-10", 300), (101, "2024-01-15", 700)]
queries = [("2024-01-01", "2024-01-10"), ("2024-01-01", "2024-01-15")]
print(process_maintenance_queries(maintenance_logs, queries))

[800, 1500]
